In [1]:
from openai import OpenAI
import config
import json
import requests

In [2]:
client = OpenAI(
api_key=config.OPENAI_API_KEY,
)
weather_api=config.WEATHER_API_KEY

In [3]:
system_message = f""" Eres un bot muy cordial, dispuesto a responder todas las solicitudes que recibas. Tus respuestas deben tener una longitud máxima de 10 palabras"""

In [4]:
messages = [
    {"role": "system", "content":system_message}
]

In [5]:
def query_chatgpt_tools(messages):
    '''Send messages to OpenAI's Chat GPT model and get a response.'''
    #You can change the model name for better performance (gpt-3.5-turbo, gpt-4 or gpt-4-1106-preview)
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125", 
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )
    response_message = response.choices[0].message
    return response_message


def query_chatgpt(messages):
    '''Send messages to OpenAI's Chat GPT model and get a response.'''
    #You can change the model name for better performance (gpt-3.5-turbo, gpt-4 or gpt-4-1106-preview)
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125", 
        messages=messages,
    )
    response_message = response.choices[0].message.content
    return response_message

In [6]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Obtener el tiempo actual en un lugar determinado",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "La ciudad, por ejemplo Madrid o Barcelona",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_product_info",
            "description": "Obtener información sobre el producto, si no se especifica, enumerar los productos disponibles",
            "parameters": {
                "type": "object",
                "properties": {
                    "product": {
                        "type": "string",
                        "description": "El nombre del producto del que se desea obtener información en caso exista",
                    },
                },
                "required": ["product"],
            },
        },
    },
]

In [6]:
def get_weather(location):
    try:
        url=f"https://api.openweathermap.org/data/2.5/weather?q={location}&units=metric&APPID={weather_api}"
        response = requests.get(url)
        data = response.json()
    
        if response.status_code == 200:
            temperature= data['main']['temp']
            return json.dumps({"location": location, "temperature": temperature, "unit": "Celsius"})
    except:
        print("Unable to get weather")
        return json.dumps({"location": location, "temperature": "No tengo información del clima en este lugar"})

In [7]:
def get_product_info(product):
    """Get products info"""
    if product == "television" or "television" in product:
        return json.dumps([{"code": "T001","name":"4k television LG","manufacturer":"LG","price":"500"},{"code": "T002","name":"4k television Samsung","manufacturer":"Samsung","price":"1000" }])
    elif product== "celular" or "celular" in product:
        return json.dumps([{"code": "C001","name":"Xiaomi Mi 13","manufacturer":"Xiaomi","price":"900"}, {"code": "C002","name":"Samsung Galaxy X","manufacturer":"Samsung","price":"1000" }])
    elif product == "tablet" or "tablet" in product:
        return json.dumps([{"code": "TA001","name":"Xiaomi Tab","manufacturer":"Xiaomi","price":"300"}, {"code": "TA002","name":"Samsung Galaxy Tab","manufacturer":"Samsung","price":"5000" }])
    else:
        return json.dumps({"message": "Parece que no tenemos ese producto disponible, actualmente tenemos televisiones, celulares y tablets." })

In [9]:
def conversation(prompt):
    global messages
    messages.append({"role": "user", "content": prompt})
    response_message = query_chatgpt_tools(messages)
    tool_calls = response_message.tool_calls

    if tool_calls:
        available_functions = {
            "get_weather":get_weather,
            "get_product_info":get_product_info,
        }
        messages.append(response_message)
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            if function_name=="get_weather":
                function_response = function_to_call(
                    location=function_args.get("location"),
                )
            elif function_name=="get_product_info":
                function_response = function_to_call(
                    product=function_args.get("product"),
                )
            messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            })
        second_response = query_chatgpt(messages)
        messages.append({"role": "assistant", "content": second_response})
        return second_response
    else:
        messages.append({"role": "assistant", "content": response_message.content})
        return response_message.content

In [10]:
conversation("Hola")

'¡Hola! ¿En qué puedo ayudarte hoy?'

In [11]:
conversation("¿En qué puedes ayudarme?")

'Puedo proporcionarte información sobre el clima y productos específicos.'

In [12]:
conversation("Genial, entonces dime el clima")

'Por favor proporciona la ciudad para ver el clima.'

In [13]:
conversation("Madrid")

'El clima en Madrid es de 19.42°C. ¡Agradable temperatura!'

In [11]:
conversation("Ok, ahora dime que productos tienes")

'Actualmente tenemos televisiones, celulares y tablets disponibles. ¿Te interesa alguno de ellos?'

In [12]:
conversation("Háblame acerca de las televisiones")

'Tenemos televisores 4k de LG y Samsung disponibles. ¡Excelentes opciones!'

In [13]:
conversation("Dame más detalles")

'Televisiones disponibles: 4k LG (500) y Samsung (1000).'

In [14]:
conversation("¿Y celulares?")

'Celulares disponibles: Xiaomi Mi 13 (900) y Samsung Galaxy X (1000).'

In [15]:
messages

[{'role': 'system',
  'content': ' Eres un bot muy cordial, dispuesto a responder todas las solicitudes que recibas. Tus respuestas deben tener una longitud máxima de 10 palabras'},
 {'role': 'user', 'content': 'Hola'},
 {'role': 'assistant', 'content': '¡Hola! ¿En qué puedo ayudarte hoy?'},
 {'role': 'user', 'content': 'Ok, ahora dime que productos tienes'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_2ZqxjsPqbCJK6jiQYd50HIIC', function=Function(arguments='{"product":"all"}', name='get_product_info'), type='function')]),
 {'tool_call_id': 'call_2ZqxjsPqbCJK6jiQYd50HIIC',
  'role': 'tool',
  'name': 'get_product_info',
  'content': '{"message": "Parece que no tenemos ese producto disponible, actualmente tenemos televisiones, celulares y tablets."}'},
 {'role': 'assistant',
  'content': 'Actualmente tenemos televisiones, celulares y tablets disponibles. ¿Te interesa alguno de ellos?'},
 {'role': 'user', 'c

In [8]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Obtener el tiempo actual en un lugar determinado",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "La ciudad, por ejemplo Madrid o Barcelona",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_product_info",
            "description": "Obtener información sobre el producto, si no se especifica, enumerar los productos disponibles",
            "parameters": {
                "type": "object",
                "properties": {
                    "product": {
                        "type": "string",
                        "description": "El nombre del producto del que se desea obtener información en caso exista",
                    },
                },
                "required": ["product"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_function_command",
            "description": "Obtener el comando respectivo basado en la acción recibida",
            "parameters": {
                "type": "object",
                "properties": {
                    "action": {
                        "type": "string",
                        "description": "La acción que se realiza",
                    },
                },
                "required": ["action"],
            },
        },
    },
]

In [9]:
def get_function_command(action):
    """Get action command"""
    return json.dumps([{"action": "relacionado a saludar","command":"[hello_world]"},{"action": "relacionado a correr","command":"[running_boy]"}])

In [10]:
def conversation(prompt):
    global messages
    messages.append({"role": "user", "content": prompt})
    response_message = query_chatgpt_tools(messages)
    tool_calls = response_message.tool_calls

    if tool_calls:
        available_functions = {
            "get_weather":get_weather,
            "get_function_command":get_function_command,
            "get_product_info":get_product_info,
        }
        messages.append(response_message)
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            if function_name=="get_weather":
                function_response = function_to_call(
                    location=function_args.get("location"),
                )
            elif function_name=="get_product_info":
                function_response = function_to_call(
                    product=function_args.get("product"),
                )
            elif function_name=="get_function_command":
                function_response = function_to_call(
                    action=function_args.get("action"),
                )
            messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            })
        second_response = query_chatgpt(messages)
        messages.append({"role": "assistant", "content": second_response})
        return second_response
    else:
        messages.append({"role": "assistant", "content": response_message.content})
        return response_message.content

In [11]:
conversation("hola")


'¡Hola! ¿En qué puedo ayudarte hoy?'

In [12]:
conversation("Dame el comando para saludar")

'El comando para saludar es: [hello_world]'

In [13]:
conversation("Dame el comando para correr")

'El comando para correr es: [running_boy] ¡Diviértete!'